# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
features = ['Was_home',
 'Team_rating',
 'Opp_rating',
 'Rating_difference',
 'Min_2',
 'Start_2',
 'Gls_2',
 'Sh_2',
 'SoT_2',
 'Ast_2',
 'CrdY_2',
 'CrdR_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'ICT_index_2',
 'FPL_points_2',
 'Baseline_points_2',
 'Bonus_2',
 'Min_4',
 'Start_4',
 'Gls_4',
 'Sh_4',
 'SoT_4',
 'Ast_4',
 'CrdY_4',
 'CrdR_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'ICT_index_4',
 'FPL_points_4',
 'Baseline_points_4',
 'Bonus_4',
 'Min_30',
 'Start_30',
 'Gls_30',
 'Sh_30',
 'SoT_30',
 'Ast_30',
 'CrdY_30',
 'CrdR_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'ICT_index_30',
 'FPL_points_30',
 'Baseline_points_30',
 'Bonus_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(features)

61

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
X_test = df[df["Season"] == CURRENT_SEASON][features]
# .reset_index(drop=True)

In [7]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Baseline_points_2',
       'Baseline_points_30', 'Baseline_points_4', 'Bonus_2', 'Bonus_30',
       'Bonus_4', 'CrdR_2', 'CrdR_30', 'CrdR_4', 'CrdY_2', 'CrdY_30', 'CrdY_4',
       'DEF', 'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GKP', 'GW', 'Gls_2', 'Gls_30', 'Gls_4',
       'ICT_index_2', 'ICT_index_30', 'ICT_index_4', 'MID', 'Min_2', 'Min_30',
       'Min_4', 'Name_original', 'Opp_rating', 'Opp_score_2', 'Opp_score_30',
       'Opp_score_4', 'Opponent', 'Rating_difference', 'Season', 'Sh_2',
       'Sh_30', 'Sh_4', 'SoT_2', 'SoT_30', 'SoT_4', 'Start_2', 'Start_30',
       'Start_4', 'Team', 'Team_CS_2', 'Team_CS_30', 'Team_CS_4',
       'Team_rating', 'Team_result_2', 'Team_result_30', 'Team_result_4',
       'Team_score_2', 'Team_score_30', 'Team_score_4', 'Was_home', 'xA_2',
       'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'Finished', 'Date', 'Price'],
      dtype='object')

In [8]:
X_test.columns

Index(['Was_home', 'Team_rating', 'Opp_rating', 'Rating_difference', 'Min_2',
       'Start_2', 'Gls_2', 'Sh_2', 'SoT_2', 'Ast_2', 'CrdY_2', 'CrdR_2',
       'xG_2', 'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2',
       'Team_result_2', 'ICT_index_2', 'FPL_points_2', 'Baseline_points_2',
       'Bonus_2', 'Min_4', 'Start_4', 'Gls_4', 'Sh_4', 'SoT_4', 'Ast_4',
       'CrdY_4', 'CrdR_4', 'xG_4', 'xA_4', 'Team_CS_4', 'Team_score_4',
       'Opp_score_4', 'Team_result_4', 'ICT_index_4', 'FPL_points_4',
       'Baseline_points_4', 'Bonus_4', 'Min_30', 'Start_30', 'Gls_30', 'Sh_30',
       'SoT_30', 'Ast_30', 'CrdY_30', 'CrdR_30', 'xG_30', 'xA_30',
       'Team_CS_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30',
       'ICT_index_30', 'FPL_points_30', 'Baseline_points_30', 'Bonus_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [9]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [10]:
model

GradientBoostingRegressor(random_state=42)

# Predicting

In [11]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [12]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [13]:
preds = get_predictions(model, df, X_test)

16792 (16792, 73)


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [15]:
# next gameweek
preds[preds["GW"] == 2].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
9488,Konstantinos Tsimikas,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,DEF,45.0,6.518966
9639,Kyle Walker,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,DEF,50.0,6.515971
16756,Zack Steffen,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,GKP,40.0,6.444765
5025,Erling Haaland,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,FWD,140.0,6.280906
8171,John Stones,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,DEF,54.0,6.133664
10585,Manuel Akanji,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,DEF,50.0,6.049752
2939,Cheikhou Kouyaté,2.0,2023-24,Nott'ham Forest,Sheffield United,1.0,1671.868774,1637.473755,MID,50.0,5.996168
3091,Chris Wood,2.0,2023-24,Nott'ham Forest,Sheffield United,1.0,1671.868774,1637.473755,FWD,50.0,5.919357
12250,Nathan Aké,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,DEF,50.0,5.374012
11229,Mateo Kovačić,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2080.850586,1887.064819,MID,50.0,5.348345


In [16]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
7589,Jean-Philippe Mateta,28.0,2023-24,Crystal Palace,Luton,1.0,1763.994995,1602.327637,FWD,50.0,7.603447
7576,Jean-Philippe Mateta,15.0,2023-24,Crystal Palace,Bournemouth,1.0,1763.994995,1662.536621,FWD,50.0,7.490715
7583,Jean-Philippe Mateta,22.0,2023-24,Crystal Palace,Sheffield United,1.0,1763.994995,1637.473755,FWD,50.0,7.422484
7569,Jean-Philippe Mateta,8.0,2023-24,Crystal Palace,Nott'ham Forest,1.0,1763.994995,1671.868774,FWD,50.0,7.342421
7573,Jean-Philippe Mateta,12.0,2023-24,Crystal Palace,Everton,1.0,1763.994995,1699.505859,FWD,50.0,7.307678
7587,Jean-Philippe Mateta,26.0,2023-24,Crystal Palace,Burnley,1.0,1763.994995,1722.802979,FWD,50.0,7.307678
7567,Jean-Philippe Mateta,6.0,2023-24,Crystal Palace,Fulham,1.0,1763.994995,1745.365723,FWD,50.0,7.307678
7565,Jean-Philippe Mateta,4.0,2023-24,Crystal Palace,Wolves,1.0,1763.994995,1715.631348,FWD,50.0,7.307678
9648,Kyle Walker,11.0,2023-24,Manchester City,Bournemouth,1.0,2080.850586,1662.536621,DEF,50.0,7.224203
9656,Kyle Walker,20.0,2023-24,Manchester City,Sheffield United,1.0,2080.850586,1637.473755,DEF,50.0,7.224203


In [17]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Kyle Walker,236.679820
Erling Haaland,232.248873
Manuel Akanji,218.458505
Zack Steffen,215.697452
Kieran Trippier,214.659150
John Stones,210.948430
Konstantinos Tsimikas,207.774446
Leandro Trossard,194.519724
Joško Gvardiol,193.845640


# Saving to file

In [18]:
preds.to_csv("predictions/preds_next_season.csv", index=False)